In [1]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xf VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!tar -xf VOCtest_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtestnoimgs_06-Nov-2007.tar

--2020-05-15 13:41:24--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  8.86MB/s    in 52s     

2020-05-15 13:42:17 (8.45 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

--2020-05-15 13:42:21--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================

In [0]:
from PIL import Image
import tensorflow as tf
import io, json, os, ray
from xml.etree import ElementTree as ET

In [0]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
num_train_shards = 2
num_val_shards = 2
num_test_shards = 2
ray.init()
tf.get_logger().setLevel('ERROR')

2020-05-15 13:45:50,680	INFO resource_spec.py:212 -- Starting Ray with 7.23 GiB memory available for workers and up to 3.62 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-15 13:45:51,154	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [0]:
def chunkify(l, n):
    size = len(l) // n
    start = 0
    results = []
    for i in range(n - 1):
        results.append(l[start:start + size])
        start += size
    results.append(l[start:])
    return results

In [0]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [0]:
def genreate_tfexample(anno):
    with open(anno['filepath'], 'rb') as image_file:
        content = image_file.read()
    width = anno['width']
    height = anno['height']
    depth = anno['depth']
    if depth != 3:
        print('WANRNING: Image {} has depth of {}'.format(
            anno['filename'], depth))
    class_ids = []
    class_texts = []
    bbox_xmins = []
    bbox_ymins = []
    bbox_xmaxs = []
    bbox_ymaxs = []
    for bbox in anno['bboxes']:
        class_ids.append(bbox['class_id'])
        class_texts.append(bbox['class_text'].encode())
        xmin, ymin, xmax, ymax = bbox['xmin'], bbox['ymin'], bbox[
            'xmax'], bbox['ymax']
        bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax = float(
            xmin) / width, float(ymin) / height, float(xmax) / width, float(
                ymax) / height
        assert bbox_xmin <= 1 and bbox_xmin >= 0
        assert bbox_ymin <= 1 and bbox_ymin >= 0
        assert bbox_xmax <= 1 and bbox_xmax >= 0
        assert bbox_ymax <= 1 and bbox_ymax >= 0
        bbox_xmins.append(bbox_xmin)
        bbox_ymins.append(bbox_ymin)
        bbox_xmaxs.append(bbox_xmax)
        bbox_ymaxs.append(bbox_ymax)
    feature = {
        'image/height':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/depth':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[depth])),
        'image/object/bbox/xmin':
        tf.train.Feature(float_list=tf.train.FloatList(value=bbox_xmins)),
        'image/object/bbox/ymin':
        tf.train.Feature(float_list=tf.train.FloatList(value=bbox_ymins)),
        'image/object/bbox/xmax':
        tf.train.Feature(float_list=tf.train.FloatList(value=bbox_xmaxs)),
        'image/object/bbox/ymax':
        tf.train.Feature(float_list=tf.train.FloatList(value=bbox_ymaxs)),
        'image/object/class/label':
        tf.train.Feature(int64_list=tf.train.Int64List(value=class_ids)),
        'image/object/class/text':
        tf.train.Feature(bytes_list=tf.train.BytesList(value=class_texts)),
        'image/encoded':
        _bytes_feature(content),
        'image/filename':
        _bytes_feature(anno['filename'].encode())
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [0]:
@ray.remote
def build_single_tfrecord(chunk, path):
    print('start to build tf records for ' + path)

    with tf.io.TFRecordWriter(path) as writer:
        for anno in chunk:
            tf_example = genreate_tfexample(anno)
            writer.write(tf_example.SerializeToString())

    print('finished building tf records for ' + path)

In [0]:
def build_tf_records(annotations, total_shards, split):
    chunks = chunkify(annotations, total_shards)
    futures = [
        # train_0001_of_0064.tfrecords
        build_single_tfrecord.remote(
            chunk, './tfrecords_voc/{}_{}_of_{}.tfrecords'.format(
                split,
                str(i + 1).zfill(4),
                str(total_shards).zfill(4),
            )) for i, chunk in enumerate(chunks)
    ]
    ray.get(futures)

In [0]:
def parse_one_xml(xml_file, names_map):
    tree = ET.parse(os.path.join('./VOCdevkit/VOC2007/Annotations', xml_file))
    root = tree.getroot()
    filename = root.find('.//filename').text
    filepath = os.path.join('./VOCdevkit/VOC2007/JPEGImages', filename)
    objects_els = root.findall('.//object')
    size_el = root.find('size')
    width = int(size_el.find('width').text)
    height = int(size_el.find('height').text)
    depth = int(size_el.find('depth').text)

    bboxes = []
    for obj_el in objects_els:
        name_el = obj_el.find('name')
        bbox_el = obj_el.find('bndbox')
        bboxes.append({
            'class_text': name_el.text,
            'class_id': names_map[name_el.text],
            'xmin': int(bbox_el.find('xmin').text),
            'ymin': int(bbox_el.find('ymin').text),
            'xmax': int(bbox_el.find('xmax').text),
            'ymax': int(bbox_el.find('ymax').text),
        })

    return {
        'filepath': filepath,
        'filename': filename,
        'width': width,
        'height': height,
        'depth': depth,
        'bboxes': bboxes,
    }

In [0]:
def main():
    print('Start to parse annotations.')
    if not os.path.exists('./tfrecords_voc'):
        os.makedirs('./tfrecords_voc')
    train_val_split = {}
    with open('./VOCdevkit/VOC2007/ImageSets/Main/train.txt') as train_fp:
        lines = train_fp.read().splitlines()
        for line in lines:
            train_val_split[line] = 'train'
    with open('./VOCdevkit/VOC2007/ImageSets/Main/val.txt') as val_fp:
        lines = val_fp.read().splitlines()
        for line in lines:
            train_val_split[line] = 'val'
    with open('./VOCdevkit/VOC2007/ImageSets/Main/test.txt') as val_fp:
        lines = val_fp.read().splitlines()
        for line in lines:
            train_val_split[line] = 'test'
    names = [
             'aeroplane',
             'bicycle',
             'bird',
             'boat',
             'bottle',
             'bus',
             'car',
             'cat',
             'chair',
             'cow',
             'diningtable',
             'dog',
             'horse',
             'motorbike',
             'person',
             'pottedplant',
             'sheep',
             'sofa',
             'train',
             'tvmonitor'
            ]
    names_map = {name: i for i, name in enumerate(names)}
    print(names_map)
    train_annotations = []
    val_annotations = []
    test_annotations = []
    for xml_file in os.listdir('./VOCdevkit/VOC2007/Annotations'):
        image_id = xml_file[:-4]
        if train_val_split[image_id] == 'train':
            train_annotations.append(parse_one_xml(xml_file, names_map))
        elif train_val_split[image_id] == 'val':
            val_annotations.append(parse_one_xml(xml_file, names_map))
        elif train_val_split[image_id] == 'test':
            test_annotations.append(parse_one_xml(xml_file, names_map))
        else:
            print('WARNING: Unwanted image id {}'.format(image_id))
    print('Start to build TF Records.')
    build_tf_records(train_annotations, num_train_shards, 'train')
    build_tf_records(val_annotations, num_val_shards, 'val')
    build_tf_records(test_annotations, num_test_shards, 'test')
    print('Successfully wrote {} annotations to TF Records.'.format(
        len(train_annotations) + len(val_annotations) + len(test_annotations)))

In [16]:
if __name__ == '__main__':
    main()

Start to parse annotations.
{'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 'diningtable': 10, 'dog': 11, 'horse': 12, 'motorbike': 13, 'person': 14, 'pottedplant': 15, 'sheep': 16, 'sofa': 17, 'train': 18, 'tvmonitor': 19}
Start to build TF Records.
(pid=230) start to build tf records for ./tfrecords_voc/train_0001_of_0002.tfrecords
(pid=229) start to build tf records for ./tfrecords_voc/train_0002_of_0002.tfrecords
(pid=229) finished building tf records for ./tfrecords_voc/train_0002_of_0002.tfrecords
(pid=230) finished building tf records for ./tfrecords_voc/train_0001_of_0002.tfrecords
(pid=230) start to build tf records for ./tfrecords_voc/val_0001_of_0002.tfrecords
(pid=229) start to build tf records for ./tfrecords_voc/val_0002_of_0002.tfrecords
(pid=230) finished building tf records for ./tfrecords_voc/val_0001_of_0002.tfrecords
(pid=229) finished building tf records for ./tfrecords_voc/val_0002_of_0002.tfrec